# Demonstrating `mobile-env`

`mobile-env` is a simple and open environment for training, testing, and evaluating autonomous coordination
approaches for wireless mobile networks.

* `mobile-env` is written in pure Python and can be installed easily via [PyPI](https://pypi.org/project/mobile-env/)
* It allows simulating various scenarios with moving users in a cellular network with multiple base stations
* `mobile-env` implements the standard [Gymnasium](https://gymnasium.farama.org/) (previously [OpenAI Gym](https://gym.openai.com/)) interface such that it can be used with all common frameworks for reinforcement learning
* `mobile-env` is not restricted to reinforcement learning approaches but can also be used with conventional control approaches or dummy benchmark algorithms
* It supports both centralized, single-agent control and distributed, multi-agent control
* It can be configured easily (e.g., adjusting number and movement of users, properties of cells, etc.)
* It is also easy to extend `mobile-env`, e.g., implementing different observations, actions, or reward

As such `mobile-env` is a simple platform to evaluate and compare different coordination approaches in a meaningful way.



**Demonstration Steps:**

This demonstration consists of the following steps (split accross separate notebooks):

1. [Intro notebook](examples\demo.ipynb): Installation and usage of `mobile-env` with dummy actions
2. [Intro notebook](examples\demo.ipynb): Configuration of `mobile-env` and adjustment of the observation space (optional)
3. [SB3 notebook](examples\sb3.ipynb): Training a single-agent reinforcement learning approach with [`stable-baselines3`](https://github.com/DLR-RM/stable-baselines3)
4. **This notebook:** Training a multi-agent reinforcement learning approach with [Ray RLlib](https://docs.ray.io/en/latest/rllib.html)

## Step 4: Multi-Agent RL with Ray RLlib

As alternative to controlling cell selection centrally for all users from a single RL agent, we can also use multi-agent RL, i.e., delegating control to multiple agents that act in parallel.
As an example, we could have each RL agent responsible for the cell selection of a single user. Then we would need as many agents as we have users.
That's what happens in the predefined multi-agent scenarios, e.g., `mobile-small-ma-v0`.

Let's use RLlib to train a multi-agent policy on the `mobile-small-ma-v0` scenario, which has three base stations and five users.

### Set up Ray RLlib

To train a multi-agent approach, we can use Ray RLlib, which supports multi-agent RL out of the box. To register the predefined multi-agent scenario with RLlib, `mobile-env` provides a wrapper `RLlibMAWrapper`. But first we need to install `mobile-env` and `ray` with RLlib:

In [1]:
# install mobile-env
!pip install -U mobile-env
# install ray RLlib
!pip install ray[rllib]==2.5.1 tensorboard

In [1]:
import gymnasium
from ray.tune.registry import register_env

# use the mobile-env RLlib wrapper for RLlib
def register(config):
    # importing mobile_env registers the included environments
    import mobile_env
    from mobile_env.wrappers.multi_agent import RLlibMAWrapper

    env = gymnasium.make("mobile-small-ma-v0")
    return RLlibMAWrapper(env)

# register the predefined scenario with RLlib
register_env("mobile-small-ma-v0", register)

### Train a PPO Multi-Agent Policy

Now, that the predefined scenario is registered with RLlib, we can configure and train a multi-agent PPO approach on the scenario with RLlib.

In [2]:
import ray


# init ray with available CPUs (and GPUs) and init ray
ray.init(
  num_cpus=2,   # change to your available number of CPUs
  include_dashboard=False,
  ignore_reinit_error=True,
  log_to_driver=False,
)

2023-07-20 19:24:07,872	INFO worker.py:1636 -- Started a local Ray instance.


Python version:,3.8.16
Ray version:,2.5.1


In [3]:
import ray.air
from ray.rllib.algorithms.ppo import PPOConfig

from ray.rllib.policy.policy import PolicySpec
from ray.tune.stopper import MaximumIterationStopper

# Create an RLlib config using multi-agent PPO on mobile-env's small scenario.
config = (
    PPOConfig()
    .environment(env="mobile-small-ma-v0")
    # Here, we configure all agents to share the same policy.
    .multi_agent(
        policies={"shared_policy": PolicySpec()},
        policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: "shared_policy",
    )
    # RLlib needs +1 CPU than configured below (for the driver/traininer?)
    .resources(num_cpus_per_worker=1)
    .rollouts(num_rollout_workers=1)
)

# Create the Trainer/Tuner and define how long to train
tuner = ray.tune.Tuner(
    "PPO",
    run_config=ray.air.RunConfig(
        # Save the training progress and checkpoints locally under the specified subfolder.
        storage_path="./results_rllib",
        # Control training length by setting the number of iterations. 1 iter = 4000 time steps by default.
        stop=MaximumIterationStopper(max_iter=10),
        checkpoint_config=ray.air.CheckpointConfig(checkpoint_at_end=True),
    ),
    param_space=config,
)

# Run training and save the result
result_grid = tuner.fit()

c:\Users\stbs\.conda\envs\mobile-env\lib\site-packages\tensorflow_probability\python\__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
2023-07-20 19:25:06,719	WARNING syncer.py:222 -- You are using remote storage, but you don't have `fsspec` installed. This can lead to inefficient syncing behavior. To avoid this, install fsspec with `pip install fsspec`. Depending on your remote storage provider, consider installing the respective fsspec-package (see https://github.com/fsspec).
2023-07-20 19:25:06,737	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/a3c.py` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
2023-07-20 19:25:06,739	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/a3c.py` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
2023-07

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_total,training_iteration,trial_id
PPO_mobile-small-ma-v0_5f1f2_00000,20000,"{'ObsPreprocessorConnector_ms': 0.009922981262207031, 'StateBufferConnector_ms': 0.017557144165039062, 'ViewRequirementAgentConnector_ms': 0.5606132745742798}","{'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 20000, 'num_agent_steps_trained': 20000}",{},2023-07-20_19-27-03,True,100,{},-19.9275,-186.81,-322.086,40,40,Lenovo-Yoga,"{'learner': {'shared_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 1.7262574494960707, 'cur_kl_coeff': 0.2, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 8.068133396302565, 'policy_loss': -0.006108309560398039, 'vf_loss': 8.072629420266283, 'vf_explained_var': 0.03875742717913002, 'kl': 0.008061309833623109, 'entropy': 1.3782111626268698, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 127.38853503184713, 'num_grad_updates_lifetime': 2355.5, 'diff_num_grad_updates_vs_sampler_policy': 2354.5}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 20000, 'num_agent_steps_trained': 20000}",1,127.0.0.1,20000,20000,4000,4000,50.4775,4000,4000,50.4775,0,1,0,0,4000,"{'cpu_util_percent': 12.044144144144147, 'ram_util_percent': 95.96396396396395}",8360,{'shared_policy': 4.675273777830706},{'shared_policy': -37.361940196467856},{'shared_policy': -88.18277678648225},"{'mean_raw_obs_processing_ms': 1.5595255420077476, 'mean_inference_ms': 1.6059204507726217, 'mean_action_processing_ms': 0.5598039038328492, 'mean_env_wait_ms': 4.779515639450276, 'mean_env_render_ms': 0.0}",79.252,79.252,79.252,"{'training_iteration_time_ms': 79243.155, 'sample_time_ms': 34188.102, 'learn_time_ms': 45049.352, 'learn_throughput': 88.792, 'synch_weights_time_ms': 2.001}",1689874023,4000,1,5f1f2_00000


2023-07-20 19:27:03,902	INFO tune.py:1111 -- Total run time: 117.18 seconds (115.89 seconds for the tuning loop).


We can check the learning curve on TensorBoard. The corresponding files are in the configured `save_dir=results_rllib`.

The "episode_reward_mean" should increase with increasing training, indicating that the agent is learning. RLlib also logs many other metrics by default, which can be useful for debugging. 

In [10]:
%load_ext tensorboard
%tensorboard --logdir results_rllib

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 17908), started 0:00:22 ago. (Use '!kill 17908' to kill it.)

### Load and Test the Trained Multi-Agent Policy

Let's load the trained multi-agent model and visualize the learned multi-agent policy:

In [6]:
from ray.rllib.algorithms.algorithm import Algorithm

# load the trained agent from the stored checkpoint
best_result = result_grid.get_best_result(metric="episode_reward_mean", mode="max")
ppo = Algorithm.from_checkpoint(best_result.checkpoint)

2023-07-20 19:29:26,871	INFO trainable.py:173 -- Trainable.setup took 14.657 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-07-20 19:29:26,881	WARNING util.py:68 -- Install gputil for GPU system monitoring.


In [11]:
import mobile_env
import matplotlib.pyplot as plt
from IPython import display

# create the env for testing
# pass rgb_array as render mode so the env can be rendered inside the notebook
env = gymnasium.make("mobile-small-ma-v0", render_mode="rgb_array")
obs, info = env.reset()
done = False

# run one episode with the trained model
while not done:
    # gather action from each actor (for each UE)
    action = {}
    for agent_id, agent_obs in obs.items():
        # compute the action for the given agent using the shared policy
        action[agent_id] = ppo.compute_single_action(agent_obs, policy_id="shared_policy")

    # apply actions and perform step on simulation environment
    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

    # render environment as RGB
    plt.imshow(env.render())
    display.display(plt.gcf())
    display.clear_output(wait=True)

{0: -0.08314402193659076, 1: -0.06218562828460161, 2: -1.0, 3: -1.0, 4: -0.15216507928783843}


While the learned policy is not yet perfect (the reward is still increasing, i.e., agent is still learning), the visualization shows that most users are connected to suitable cells.

Feel free to experiment more with `mobile-env`, e.g., by training longer or customizing the environment.
The [documentation](https://mobile-env.readthedocs.io/en/latest/) provides further information about the API.
If you still have open questions or run into issues, you can [open an issue on GitHub](https://github.com/stefanbschneider/mobile-env/issues).

We hope `mobile-env` is useful for you. If you use `mobile-env`, please cite it and let us know - then we can list your work on `mobile-env`'s Readme.
We also very much appreciate contributions in the form of pull requests.

```
@inproceedings{schneider2022mobileenv,
  author = {Schneider, Stefan and Werner, Stefan and Khalili, Ramin and Hecker, Artur and Karl, Holger},
  title = {mobile-env: An Open Platform for Reinforcement Learning in Wireless Mobile Networks},
  booktitle={Network Operations and Management Symposium (NOMS)},
  year = {2022},
  publisher = {IEEE/IFIP},
}
```

Happy training/learning!